<a href="https://colab.research.google.com/github/keiverdatascience/labo2025v/blob/main/src/entrega_final/competicion_kaggle_gerencial_KN_EF1003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Competición Kaggle - Modalidad Gerencial


A través del siguiente notebook para la modalidad gerencial, se propone un modelo que tiene como objetivo  predecir que clientes de Paquete Premium de la foto al 30-septiembre-2021 se darán de baja durante noviembre-2021, es decir predecir las BAJA+2 de la foto de 202109.

# Modelo de predicción - modalidad gerencial

## 1.1  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive', force_remount= True)

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab
*  Copiar en drive el archivo indice-FAPCE y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="Indice-FACPCE.xlsx"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


## 1.2 Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Thu Nov 27 09:41:44 PM 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660442,35.3,1454494,77.7,1437474,76.8
Vcells,1226661,9.4,8388608,64.0,1975136,15.1


Instalo paquete R.utils

In [3]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: data.table

Loading required package: R.utils

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.utils’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.




#### Semilla primigenia y N° experimento
No olvide modificar su semilla primigenia y el nombre del experimento se quiere tener trazabilidad de los resultados

In [4]:
PARAM <- list()
PARAM$semilla_primigenia <- 245681

PARAM$experimento <- 1003
PARAM$dataset <- "gerencial_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [5]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("EF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## 1.3   Preprocesamiento del dataset

#### 1.3.1 DT incorporar dataset

In [6]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

In [ ]:
colnames(dataset)

[1] "numero_de_cliente"           "foto_mes"                   
 [3] "internet"                    "cliente_edad"               
 [5] "cliente_antiguedad"          "mrentabilidad"              
 [7] "mrentabilidad_annual"        "mcomisiones"                
 [9] "mactivos_margen"             "mpasivos_margen"            
[11] "cproductos"                  "mcuenta_corriente"          
[13] "mcaja_ahorro"                "cdescubierto_preacordado"   
[15] "mcuentas_saldo"              "ctarjeta_visa_transacciones"
[17] "mtarjeta_visa_consumo"       "mtarjeta_master_consumo"    
[19] "mprestamos_personales"       "cpayroll_trx"               
[21] "mpayroll"                    "ccomisiones_mantenimiento"  
[23] "ccallcenter_transacciones"   "chomebanking_transacciones" 
[25] "ctrx_quarter"                "Master_status"              
[27] "Master_fechaalta"            "Master_mpagominimo"         
[29] "Visa_status"                 "Visa_fechaalta"             
[31] "Visa_mpagominimo"            "clase_ternaria"

#### 1.3.2 Rápido y breve EDA para explorar los datos. Este paso lo puede omitir si es una persona ansiosa

In [ ]:
if( !require("DataExplorer")) install.packages("DataExplorer")
require("DataExplorer")

create_report(dataset, output_file = "EDA_automatico.html")



processing file: report.rmd



1/42                                 
2/42 [global_options]                
3/42                                 
4/42 [introduce]                     
5/42                                 
6/42 [plot_intro]                    
7/42                                 
8/42 [data_structure]                
9/42                                 
10/42 [missing_profile]               
11/42                                 
12/42 [univariate_distribution_header]
13/42                                 
14/42 [plot_histogram]                
15/42                                 
16/42 [plot_density]                  
17/42                                 
18/42 [plot_frequency_bar]            
19/42                                 
20/42 [plot_response_bar]             
21/42                                 
22/42 [plot_with_bar]                 
23/42                                 
24/42 [plot_normal_qq]                
25/42                                 
26/42 [plot_response_qq]          

output file: /content/.drive/My Drive/labo1/exp/EF1000/report.knit.md




/usr/bin/pandoc +RTS -K512m -RTS '/content/.drive/My Drive/labo1/exp/EF1000/report.knit.md' --to html4 --from markdown+autolink_bare_uris+tex_math_single_backslash --output pandocaf732284101.html --lua-filter /usr/lib/R/site-library/rmarkdown/rmarkdown/lua/pagebreak.lua --lua-filter /usr/lib/R/site-library/rmarkdown/rmarkdown/lua/latex-div.lua --self-contained --variable bs3=TRUE --section-divs --table-of-contents --toc-depth 6 --template /usr/lib/R/site-library/rmarkdown/rmd/h/default.html --no-highlight --variable highlightjs=1 --variable theme=yeti --mathjax --variable 'mathjax-url=https://mathjax.rstudio.com/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML' --include-in-header /tmp/Rtmp8HiB85/rmarkdown-straf742890cef.html 



Output created: EDA_automatico.html



#### 1.3.3  CA  Catastrophe Analysis - Reparar variables rotas
Se intentan reparar las variables que para un mes están con todos los valores en cero.

In [7]:
head(dataset[foto_mes == 202006])

numero_de_cliente,foto_mes,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,⋯,ccallcenter_transacciones,chomebanking_transacciones,ctrx_quarter,Master_status,Master_fechaalta,Master_mpagominimo,Visa_status,Visa_fechaalta,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>
29186441,202006,0,61,283,0,0,0,0,0,⋯,0,0,32,0,6537,0.00,0,7701,856.29,CONTINUA
29187961,202006,0,61,313,0,0,0,0,0,⋯,0,0,170,0,5969,2216.97,0,8120,9782.82,CONTINUA
29193101,202006,0,67,366,0,0,0,0,0,⋯,0,0,113,0,2056,4375.29,0,2056,1372.41,CONTINUA
29193281,202006,0,54,159,0,0,0,0,0,⋯,0,0,80,0,3155,2087.94,0,3380,2017.56,CONTINUA
29198891,202006,0,54,313,0,0,0,0,0,⋯,0,0,22,0,7232,457.47,0,7272,1642.20,CONTINUA
29201701,202006,0,43,10,0,0,0,0,0,⋯,0,0,36,0,22,0.00,0,22,0.00,CONTINUA


El método que se utiliza es **Machine Learning** se asigna NA also valores.

In [8]:
dataset[ foto_mes==202006, internet:=NA]
dataset[ foto_mes==202006, mrentabilidad:=NA]
dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
dataset[ foto_mes==202006, mcomisiones:=NA]
dataset[ foto_mes==202006, mactivos_margen:=NA]
dataset[ foto_mes==202006, mpasivos_margen:=NA]
dataset[ foto_mes==202006, mcuentas_saldo:=NA]
dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

#### 1.3.3  DR  Data Drifting
Se intenta corregir el data drifting, ajustado por FACPCE

In [9]:
if( !require("readxl")) install.packages("readxl")
require("readxl")
if( !require("data.table")) install.packages("data.table")
require("data.table")
if( !require("dplyr")) install.packages("dplyr")
require("dplyr")

fapce <- read_excel("/content/datasets/Indice-FACPCE.xlsx", col_types = c("text", "text", "text")) %>%
  mutate(
    # ---- FOTO_MES CORREGIDO ----
    # Eliminar el último caracter: "2020050" → "202005"
    foto_mes = gsub("[^0-9]", "", foto_mes),
    foto_mes = substr(foto_mes, 1, 6),

    # ---- IPC ----
    #IPC = gsub("[^0-9]", "", IPC),
    IPC = as.numeric(IPC),

    # ---- coeficiente ajustes ----
    coeficiente_de_ajuste = gsub(",", ".", coeficiente_de_ajuste),
    coeficiente_de_ajuste = gsub("[^0-9.]", "", coeficiente_de_ajuste),
    coeficiente_de_ajuste = as.numeric(coeficiente_de_ajuste)
  ) %>%
  as.data.table()



Loading required package: readxl

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [10]:
  head(fapce)

foto_mes,IPC,coeficiente_de_ajuste
<chr>,<dbl>,<dbl>
202005,314.9,1.678311
202006,322.0,1.641304
202007,328.2,1.610299
202008,337.1,1.567784
202009,346.6,1.524812
202010,359.7,1.469280


In [11]:
dataset <- as.data.table(dataset)

# asegurar foto_mes como texto tipo "202005"
dataset[, foto_mes := as.character(foto_mes)]
dataset[, foto_mes := gsub("[^0-9]", "", foto_mes)]
dataset[, foto_mes := substr(foto_mes, 1, 6)]

In [12]:
cols_borrar <- c("coeficiente_de_ajuste", "coeficiente_de_ajuste.x", "coeficiente_de_ajuste.y")

for (col in cols_borrar) {
  if (col %in% names(dataset)) dataset[, (col) := NULL]
}

dataset <- merge(
  dataset,
  fapce[, .(foto_mes, coeficiente_de_ajuste)],
  by = "foto_mes",
  all.x = TRUE
)

In [13]:
# MERGE POR FOTO_MES
vars_ajustar <- c(
  "mrentabilidad",
  "mrentabilidad_annual",
  "mcomisiones",
  "mactivos_margen",
  "mpasivos_margen",
  "mcuenta_corriente",
  "mcaja_ahorro",
  "mcuentas_saldo",
  "mtarjeta_visa_consumo",
  "mtarjeta_master_consumo",
  "mprestamos_personales",
  "mpayroll",
  "Master_mpagominimo",
  "Visa_mpagominimo"
)

dataset[, (vars_ajustar) := lapply(.SD, as.numeric), .SDcols = vars_ajustar]

In [14]:
dataset[
  ,
  (vars_ajustar) := lapply(.SD, function(x) x * coeficiente_de_ajuste),
  .SDcols = vars_ajustar
]

In [15]:
head(dataset)

foto_mes,numero_de_cliente,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,⋯,chomebanking_transacciones,ctrx_quarter,Master_status,Master_fechaalta,Master_mpagominimo,Visa_status,Visa_fechaalta,Visa_mpagominimo,clase_ternaria,coeficiente_de_ajuste
<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<dbl>
202005,29186441,1,61,282,3952.3207,73445.539,2823.1534,-233.31874,1195.9641,⋯,20,47,0,6507,0.0000,0,7671,1752.10589,CONTINUA,1.678311
202005,29187961,1,61,312,8502.8416,79330.938,6504.4940,-5934.55654,7638.1760,⋯,228,232,0,5939,3386.0923,0,8090,12579.72656,CONTINUA,1.678311
202005,29193101,1,67,365,8071.7004,63770.196,11719.1896,-4971.55872,1862.0185,⋯,133,149,0,2026,4764.1531,0,2026,2775.80821,CONTINUA,1.678311
202005,29193281,1,54,158,5000.5599,148424.064,5622.3572,-1524.82907,994.7347,⋯,10,115,0,3125,1417.4340,0,3350,3405.77887,CONTINUA,1.678311
202005,29198891,1,54,312,5049.5330,37874.972,459.3200,-1080.61383,4993.8467,⋯,124,32,0,7202,393.7317,0,7242,2342.70338,CONTINUA,1.678311
202005,29205441,1,72,6,661.5732,2698.606,267.6738,-16.64884,352.4620,⋯,41,32,0,157,0.0000,0,157,98.43292,CONTINUA,1.678311


#### 1.3.4  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [16]:
dataset[, foto_mes := as.numeric(foto_mes)]

In [17]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

# el mes 1,2, ..12
if( atributos_presentes( c("foto_mes") ))
  dataset[, kmes := foto_mes %% 100]

# variable extraida de una tesis de maestria de Irlanda
if( atributos_presentes( c("mpayroll", "cliente_edad") ))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]


In [18]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

[1] "foto_mes"                    "numero_de_cliente"          
 [3] "internet"                    "cliente_edad"               
 [5] "cliente_antiguedad"          "mrentabilidad"              
 [7] "mrentabilidad_annual"        "mcomisiones"                
 [9] "mactivos_margen"             "mpasivos_margen"            
[11] "cproductos"                  "mcuenta_corriente"          
[13] "mcaja_ahorro"                "cdescubierto_preacordado"   
[15] "mcuentas_saldo"              "ctarjeta_visa_transacciones"
[17] "mtarjeta_visa_consumo"       "mtarjeta_master_consumo"    
[19] "mprestamos_personales"       "cpayroll_trx"               
[21] "mpayroll"                    "ccomisiones_mantenimiento"  
[23] "ccallcenter_transacciones"   "chomebanking_transacciones" 
[25] "ctrx_quarter"                "Master_status"              
[27] "Master_fechaalta"            "Master_mpagominimo"         
[29] "Visa_status"                 "Visa_fechaalta"             
[31] "Visa_mpagominimo"            "clase_ternaria"             
[33] "coeficiente_de_ajuste"       "kmes"                       
[35] "mpayroll_sobre_edad"

In [19]:
# Creación de Nuevas Variables

rank_by_month <- function(x) {
  x / (1 + abs(median(x, na.rm = TRUE)))
}

dataset[, disengagement_score :=
          -rank_by_month(ctrx_quarter) +
          -rank_by_month(mcuentas_saldo) +
          -rank_by_month(Visa_mpagominimo) +
          -rank_by_month(mpayroll_sobre_edad),
        by = foto_mes]

dataset[, revenue_stress :=
          -mcuentas_saldo +
          -Visa_mpagominimo]

In [20]:
head(dataset)

foto_mes,numero_de_cliente,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,⋯,Master_mpagominimo,Visa_status,Visa_fechaalta,Visa_mpagominimo,clase_ternaria,coeficiente_de_ajuste,kmes,mpayroll_sobre_edad,disengagement_score,revenue_stress
<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<int>,<int>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
202005,29186441,1,61,282,3952.3207,73445.539,2823.1534,-233.31874,1195.9641,⋯,0.0000,0,7671,1752.10589,CONTINUA,1.678311,5,0.000,-3.390416,-102703.75
202005,29187961,1,61,312,8502.8416,79330.938,6504.4940,-5934.55654,7638.1760,⋯,3386.0923,0,8090,12579.72656,CONTINUA,1.678311,5,0.000,-23.014203,-731053.84
202005,29193101,1,67,365,8071.7004,63770.196,11719.1896,-4971.55872,1862.0185,⋯,4764.1531,0,2026,2775.80821,CONTINUA,1.678311,5,0.000,-7.980975,-258322.64
202005,29193281,1,54,158,5000.5599,148424.064,5622.3572,-1524.82907,994.7347,⋯,1417.4340,0,3350,3405.77887,CONTINUA,1.678311,5,0.000,-10.364532,-390682.44
202005,29198891,1,54,312,5049.5330,37874.972,459.3200,-1080.61383,4993.8467,⋯,393.7317,0,7242,2342.70338,CONTINUA,1.678311,5,0.000,-8.470377,-371415.33
202005,29205441,1,72,6,661.5732,2698.606,267.6738,-16.64884,352.4620,⋯,0.0000,0,157,98.43292,CONTINUA,1.678311,5,1519.913,-27.402994,-70262.61


#### 1.3.4 Feature engineering RF

In [21]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

Loading required package: lightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lightgbm’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: lightgbm



In [22]:
AgregaVarRandomForest <- function() {

  cat( "inicio AgregaVarRandomForest()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% PARAM$FE_rf$train$clase01_valor1,
      clase01 := 1L ]

  campos_buenos <- setdiff(
    colnames(dataset),
    c( "clase_ternaria", "clase01")
  )

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$FE_rf$train$training )]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    free_raw_data = FALSE
  )

  modelo <- lgb.train(
     data = dtrain,
     param = PARAM$FE_rf$lgb_param,
     verbose = -100
  )

  cat( "Fin construccion RandomForest\n" )
  # grabo el modelo, achivo .model
  lgb.save(modelo, file="modelo.model" )

  qarbolitos <- copy(PARAM$FE_rf$lgb_param$num_iterations)

  periodos <- dataset[ , unique( foto_mes ) ]

  for( periodo in  periodos )
  {
    cat( "periodo = ", periodo, "\n" )
    datamatrix <- data.matrix(dataset[ foto_mes== periodo, campos_buenos, with = FALSE])

    cat( "Inicio prediccion\n" )
    prediccion <- predict(
        modelo,
        datamatrix,
        type = "leaf"
    )
    cat( "Fin prediccion\n" )

    for( arbolito in 1:qarbolitos )
    {
       cat( arbolito, " " )
       hojas_arbol <- unique(prediccion[ , arbolito])

       for (pos in 1:length(hojas_arbol)) {
         # el numero de nodo de la hoja, estan salteados
         nodo_id <- hojas_arbol[pos]
         dataset[ foto_mes== periodo, paste0(
            "rf_", sprintf("%03d", arbolito),
             "_", sprintf("%03d", nodo_id)
          ) :=  as.integer( nodo_id == prediccion[ , arbolito]) ]

       }

       rm( hojas_arbol )
    }
    cat( "\n" )

    rm( prediccion )
    rm( datamatrix )
    gc(verbose= FALSE)
  }

  gc(verbose= FALSE)

  # borro clase01 , no debe ensuciar el dataset
  dataset[ , clase01 := NULL ]

}


In [23]:
# Parametros de Feature Engineering  a partir de hojas de Random Forest

# Estos CUATRO parametros son los que se deben modificar
PARAM$FE_rf$arbolitos= 10
PARAM$FE_rf$hojas_por_arbol= 8
PARAM$FE_rf$datos_por_hoja= 150
PARAM$FE_rf$mtry_ratio= 0.2

# Estos son quasi fijos
PARAM$FE_rf$train$clase01_valor1 <- c( "BAJA+2", "BAJA+1")
PARAM$FE_rf$train$training <- c( 202101, 202102, 202103)

# Estos TAMBIEN son quasi fijos
PARAM$FE_rf$lgb_param <-list(
    # parametros que se pueden cambiar
    num_iterations = PARAM$FE_rf$arbolitos,
    num_leaves  = PARAM$FE_rf$hojas_por_arbol,
    min_data_in_leaf = PARAM$FE_rf$datos_por_hoja,
    feature_fraction_bynode  = PARAM$FE_rf$mtry_ratio,

    # para que LightGBM emule Random Forest
    boosting = "rf",
    bagging_fraction = ( 1.0 - 1.0/exp(1.0) ),
    bagging_freq = 1.0,
    feature_fraction = 1.0,

    # genericos de LightGBM
    max_bin = 31L,
    objective = "binary",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    force_row_wise = TRUE,
    verbosity = -100,
    max_depth = -1L,
    min_gain_to_split = 0.0,
    min_sum_hessian_in_leaf = 0.001,
    lambda_l1 = 0.0,
    lambda_l2 = 0.0,

    pos_bagging_fraction = 1.0,
    neg_bagging_fraction = 1.0,
    is_unbalance = FALSE,
    scale_pos_weight = 1.0,

    drop_rate = 0.1,
    max_drop = 50,
    skip_drop = 0.5,

    extra_trees = FALSE
  )

In [24]:
# Feature Engineering agregando variables de Random Forest
AgregaVarRandomForest()

inicio AgregaVarRandomForest()
Fin construccion RandomForest
periodo =  202005 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  
periodo =  202006 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  
periodo =  202007 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  
periodo =  202008 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  
periodo =  202009 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  
periodo =  202010 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  
periodo =  202011 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  
periodo =  202012 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  
periodo =  202101 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  
periodo =  202102 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  
periodo =  202103 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  
periodo =  2021

In [25]:
ncol(dataset)
colnames(dataset)

[1] 118

[1] "foto_mes"                    "numero_de_cliente"          
  [3] "internet"                    "cliente_edad"               
  [5] "cliente_antiguedad"          "mrentabilidad"              
  [7] "mrentabilidad_annual"        "mcomisiones"                
  [9] "mactivos_margen"             "mpasivos_margen"            
 [11] "cproductos"                  "mcuenta_corriente"          
 [13] "mcaja_ahorro"                "cdescubierto_preacordado"   
 [15] "mcuentas_saldo"              "ctarjeta_visa_transacciones"
 [17] "mtarjeta_visa_consumo"       "mtarjeta_master_consumo"    
 [19] "mprestamos_personales"       "cpayroll_trx"               
 [21] "mpayroll"                    "ccomisiones_mantenimiento"  
 [23] "ccallcenter_transacciones"   "chomebanking_transacciones" 
 [25] "ctrx_quarter"                "Master_status"              
 [27] "Master_fechaalta"            "Master_mpagominimo"         
 [29] "Visa_status"                 "Visa_fechaalta"             
 [31] "Visa_mpagominimo"            "clase_ternaria"             
 [33] "coeficiente_de_ajuste"       "kmes"                       
 [35] "mpayroll_sobre_edad"         "disengagement_score"        
 [37] "revenue_stress"              "entrenamiento"              
 [39] "rf_001_007"                  "rf_001_002"                 
 [41] "rf_001_000"                  "rf_001_001"                 
 [43] "rf_001_005"                  "rf_001_004"                 
 [45] "rf_001_003"                  "rf_001_006"                 
 [47] "rf_002_003"                  "rf_002_000"                 
 [49] "rf_002_004"                  "rf_002_007"                 
 [51] "rf_002_006"                  "rf_002_002"                 
 [53] "rf_002_005"                  "rf_002_001"                 
 [55] "rf_003_005"                  "rf_003_006"                 
 [57] "rf_003_001"                  "rf_003_000"                 
 [59] "rf_003_007"                  "rf_003_002"                 
 [61] "rf_003_004"                  "rf_003_003"                 
 [63] "rf_004_001"                  "rf_004_006"                 
 [65] "rf_004_002"                  "rf_004_000"                 
 [67] "rf_004_005"                  "rf_004_007"                 
 [69] "rf_004_003"                  "rf_004_004"                 
 [71] "rf_005_001"                  "rf_005_006"                 
 [73] "rf_005_003"                  "rf_005_005"                 
 [75] "rf_005_000"                  "rf_005_007"                 
 [77] "rf_005_002"                  "rf_005_004"                 
 [79] "rf_006_001"                  "rf_006_002"                 
 [81] "rf_006_007"                  "rf_006_000"                 
 [83] "rf_006_004"                  "rf_006_003"                 
 [85] "rf_006_005"                  "rf_006_006"                 
 [87] "rf_007_000"                  "rf_007_005"                 
 [89] "rf_007_004"                  "rf_007_003"                 
 [91] "rf_007_002"                  "rf_007_007"                 
 [93] "rf_007_001"                  "rf_007_006"                 
 [95] "rf_008_004"                  "rf_008_005"                 
 [97] "rf_008_001"                  "rf_008_003"                 
 [99] "rf_008_000"                  "rf_008_006"                 
[101] "rf_008_002"                  "rf_008_007"                 
[103] "rf_009_007"                  "rf_009_003"                 
[105] "rf_009_006"                  "rf_009_001"                 
[107] "rf_009_004"                  "rf_009_002"                 
[109] "rf_009_000"                  "rf_009_005"                 
[111] "rf_010_005"                  "rf_010_007"                 
[113] "rf_010_001"                  "rf_010_000"                 
[115] "rf_010_004"                  "rf_010_006"                 
[117] "rf_010_003"                  "rf_010_002"

#### 1.3.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

##### Probando con deltas y  lags

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [26]:
# Feature Engineering Historico

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Verificacion de los campos recien creados

In [27]:
ncol(dataset)
colnames(dataset)

[1] 578

[1] "foto_mes"                           "numero_de_cliente"                 
  [3] "internet"                           "cliente_edad"                      
  [5] "cliente_antiguedad"                 "mrentabilidad"                     
  [7] "mrentabilidad_annual"               "mcomisiones"                       
  [9] "mactivos_margen"                    "mpasivos_margen"                   
 [11] "cproductos"                         "mcuenta_corriente"                 
 [13] "mcaja_ahorro"                       "cdescubierto_preacordado"          
 [15] "mcuentas_saldo"                     "ctarjeta_visa_transacciones"       
 [17] "mtarjeta_visa_consumo"              "mtarjeta_master_consumo"           
 [19] "mprestamos_personales"              "cpayroll_trx"                      
 [21] "mpayroll"                           "ccomisiones_mantenimiento"         
 [23] "ccallcenter_transacciones"          "chomebanking_transacciones"        
 [25] "ctrx_quarter"                       "Master_status"                     
 [27] "Master_fechaalta"                   "Master_mpagominimo"                
 [29] "Visa_status"                        "Visa_fechaalta"                    
 [31] "Visa_mpagominimo"                   "clase_ternaria"                    
 [33] "coeficiente_de_ajuste"              "kmes"                              
 [35] "mpayroll_sobre_edad"                "disengagement_score"               
 [37] "revenue_stress"                     "entrenamiento"                     
 [39] "rf_001_007"                         "rf_001_002"                        
 [41] "rf_001_000"                         "rf_001_001"                        
 [43] "rf_001_005"                         "rf_001_004"                        
 [45] "rf_001_003"                         "rf_001_006"                        
 [47] "rf_002_003"                         "rf_002_000"                        
 [49] "rf_002_004"                         "rf_002_007"                        
 [51] "rf_002_006"                         "rf_002_002"                        
 [53] "rf_002_005"                         "rf_002_001"                        
 [55] "rf_003_005"                         "rf_003_006"                        
 [57] "rf_003_001"                         "rf_003_000"                        
 [59] "rf_003_007"                         "rf_003_002"                        
 [61] "rf_003_004"                         "rf_003_003"                        
 [63] "rf_004_001"                         "rf_004_006"                        
 [65] "rf_004_002"                         "rf_004_000"                        
 [67] "rf_004_005"                         "rf_004_007"                        
 [69] "rf_004_003"                         "rf_004_004"                        
 [71] "rf_005_001"                         "rf_005_006"                        
 [73] "rf_005_003"                         "rf_005_005"                        
 [75] "rf_005_000"                         "rf_005_007"                        
 [77] "rf_005_002"                         "rf_005_004"                        
 [79] "rf_006_001"                         "rf_006_002"                        
 [81] "rf_006_007"                         "rf_006_000"                        
 [83] "rf_006_004"                         "rf_006_003"                        
 [85] "rf_006_005"                         "rf_006_006"                        
 [87] "rf_007_000"                         "rf_007_005"                        
 [89] "rf_007_004"                         "rf_007_003"                        
 [91] "rf_007_002"                         "rf_007_007"                        
 [93] "rf_007_001"                         "rf_007_006"                        
 [95] "rf_008_004"                         "rf_008_005"                        
 [97] "rf_008_001"                         "rf_008_003"                        
 [99] "rf_008_000"                         "rf_008_006"                        
[1

##### Probando con EMA

In [ ]:
# Feature Engineering Historico

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

cols_promedio <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria","cliente_edad", "cliente_antiguedad", "internet", "cdescubierto_preacordado", "thomebanking",
     "Master_fultimo_cierre", "Master_fechaalta", "Master_status", "Visa_status", "Visa_fultimo_cierre", "Visa_fechaalta")
) )


# Probando EMA

# función general para EMA(N)
emaN <- function(x, n) {
  alpha <- 2 / (n + 1)

  # si todos NA -> devolver NA
  if (all(is.na(x))) return(rep(NA_real_, length(x)))

  # primer EMA = primer valor no NA
  first_idx <- which(!is.na(x))[1]
  ema_vec <- rep(NA_real_, length(x))
  ema_vec[first_idx] <- x[first_idx]

  # recursión
  if (first_idx < length(x)) {
    for (i in (first_idx+1):length(x)) {
      ema_vec[i] <- alpha * x[i] + (1 - alpha) * ema_vec[i - 1]
    }
  }

  return(ema_vec)
}


# aplicar EMA3, EMA6 y EMA12 a todas las columnas de cols_promedio
dataset[
  , c(
      paste0(cols_promedio, "_ema3"),
      paste0(cols_promedio, "_ema6"),
      paste0(cols_promedio, "_ema12")
    ) := c(
      lapply(.SD, function(col) emaN(col, 3)),
      lapply(.SD, function(col) emaN(col, 6)),
      lapply(.SD, function(col) emaN(col, 12))
    ),
  by = numero_de_cliente,
  .SDcols = cols_promedio
]

# creo ratio ult 12 meses vs 6 meses, ult 6 vs ult 3 y actuales vs 3
for (vcol in cols_promedio)
{
    ema6 <- paste0(vcol, "_ema6")
    ema3 <- paste0(vcol, "_ema3")
    ema12 <- paste0(vcol, "_ema12")

    # Ratio EMA3 / EMA6  (aceleración)
    dataset[, paste0(vcol, "ema3_ema6_ratio") :=
              fifelse(get(ema6) == 0 | is.na(get(ema6)),
                      NA_real_, get(ema3) / get(ema6))]

    # Ratio actual / EMA6 - Cambios a mediano plazo
    dataset[, paste0(vcol, "actual_ema6_ratio") :=
              fifelse(get(ema6) == 0 | is.na(get(ema6)),
                      NA_real_, get(vcol) / get(ema6))]


    # Ratio actual / EMA3 - Cambios a corto plazo
    dataset[, paste0(vcol, "actual_ema3_ratio") :=
              fifelse(get(ema3) == 0 | is.na(get(ema3)),
                      NA_real_, get(vcol) / get(ema3))]

    # Ratio actual / EMA12 - Cambios a largo plazo
        dataset[, paste0(vcol, "actual_ema12_ratio") :=
              fifelse(get(ema12) == 0 | is.na(get(ema12)),
                      NA_real_, get(vcol) / get(ema12))]
}

#Elimino emas y dejo solo los ratios
cols_a_borrar <- grep("(_ema3|_ema6|_ema12)$", names(dataset), value = TRUE)
dataset[, (cols_a_borrar) := NULL]

In [ ]:
ncol(dataset)
colnames(dataset)

[1] 546

[1] "foto_mes"                                     
  [2] "numero_de_cliente"                            
  [3] "internet"                                     
  [4] "cliente_edad"                                 
  [5] "cliente_antiguedad"                           
  [6] "mrentabilidad"                                
  [7] "mrentabilidad_annual"                         
  [8] "mcomisiones"                                  
  [9] "mactivos_margen"                              
 [10] "mpasivos_margen"                              
 [11] "cproductos"                                   
 [12] "mcuenta_corriente"                            
 [13] "mcaja_ahorro"                                 
 [14] "cdescubierto_preacordado"                     
 [15] "mcuentas_saldo"                               
 [16] "ctarjeta_visa_transacciones"                  
 [17] "mtarjeta_visa_consumo"                        
 [18] "mtarjeta_master_consumo"                      
 [19] "mprestamos_personales"                        
 [20] "cpayroll_trx"                                 
 [21] "mpayroll"                                     
 [22] "ccomisiones_mantenimiento"                    
 [23] "ccallcenter_transacciones"                    
 [24] "chomebanking_transacciones"                   
 [25] "ctrx_quarter"                                 
 [26] "Master_status"                                
 [27] "Master_fechaalta"                             
 [28] "Master_mpagominimo"                           
 [29] "Visa_status"                                  
 [30] "Visa_fechaalta"                               
 [31] "Visa_mpagominimo"                             
 [32] "clase_ternaria"                               
 [33] "coeficiente_de_ajuste"                        
 [34] "kmes"                                         
 [35] "mpayroll_sobre_edad"                          
 [36] "disengagement_score"                          
 [37] "revenue_stress"                               
 [38] "entrenamiento"                                
 [39] "rf_001_007"                                   
 [40] "rf_001_002"                                   
 [41] "rf_001_000"                                   
 [42] "rf_001_001"                                   
 [43] "rf_001_005"                                   
 [44] "rf_001_004"                                   
 [45] "rf_001_003"                                   
 [46] "rf_001_006"                                   
 [47] "rf_002_003"                                   
 [48] "rf_002_000"                                   
 [49] "rf_002_004"                                   
 [50] "rf_002_007"                                   
 [51] "rf_002_006"                                   
 [52] "rf_002_002"                                   
 [53] "rf_002_005"                                   
 [54] "rf_002_001"                                   
 [55] "rf_003_005"                                   
 [56] "rf_003_006"                                   
 [57] "rf_003_001"                                   
 [58] "rf_003_000"                                   
 [59] "rf_003_007"                                   
 [60] "rf_003_002"                                   
 [61] "rf_003_004"                                   
 [62] "rf_003_003"                                   
 [63] "rf_004_001"                                   
 [64] "rf_004_006"                                   
 [65] "rf_004_002"                                   
 [66] "rf_004_000"                                   
 [67] "rf_004_005"                                   
 [68] "rf_004_007"                                   
 [69] "rf_004_003"                                   
 [70] "rf_004_004"                                   
 [71] "rf_005_001"                                   
 [72] "rf_005_006"                                   
 [73] "rf_005_003"                                   
 [74] "rf_005_005"                                   
 [75] 

## 1.4 Modelado

### 1.4.1 Training Strategy

Luego de revisados los resultados de los experimentos y dado que esta notebook se dirige a la modalidad gerencial, no se elimina el mes de la pandemia y además no se realiza ningún undersampling, y se entrena con todos los meses del dataset.



Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 202005, 202107 ]  SIN undersampling

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 202005, 202106 ]  donde se consideran el 100% de los CONTINUA

In [28]:
PARAM$trainingstrategy$validate <- c(202107)

PARAM$trainingstrategy$training <- c(
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

PARAM$trainingstrategy$training_pct <- 1.0


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [29]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [30]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [31]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

In [32]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

[1] 16551

###  1.4.2 Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve. No se optimiza la función de ganancia tal como lo sugirió el profesor en el código original y como resultado de los experimentos hechos por la cohorte.


* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31  
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [33]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘DiceKriging’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: DiceKriging

Loading required package: mlrMBO

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘mlrMBO’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘XML’, ‘fastmatch’, ‘RcppArmadillo’, ‘mlr’, ‘ParamHelpers’, ‘smoof’, ‘BBmisc’, ‘checkmate’, ‘lhs’, ‘parallelMap’


Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers


Attaching package: ‘ParamHelpers’


The following object is masked from ‘package:R.utils’:

    isVector



Attaching package: ‘mlr’


The following objects are masked from ‘package

Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [34]:
# valor ridiculamente bajo para que corra rapido en el aula y no molestar a la *Modalidad Gerencial*
PARAM$hipeparametertuning$num_interations <- 10

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  max_bin= 31,
  learning_rate= 0.03,
  feature_fraction= 0.5,
  num_iterations= 2048,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 200
)

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower = 2L, upper = 256L),
  makeIntegerParam("min_data_in_leaf", lower = 2L, upper = 8192L)
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [35]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

Seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [36]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [37]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.

Thu Nov 27 10:04:20 PM 2025 AUC 0.927737911251474

Thu Nov 27 10:08:25 PM 2025 AUC 0.928853581847124

Thu Nov 27 10:12:40 PM 2025 AUC 0.930728557228225

Thu Nov 27 10:21:15 PM 2025 AUC 0.932900581353588

Thu Nov 27 10:25:18 PM 2025 AUC 0.93317422766168

Thu Nov 27 10:36:51 PM 2025 AUC 0.935447044457677

Thu Nov 27 10:42:37 PM 2025 AUC 0.925118692058696

Thu Nov 27 10:46:33 PM 2025 AUC 0.929330435447508

[mbo] 0: num_leaves=200; min_data_in_leaf=7679 : y = 0.928 : 317.8 secs : initdesign

[mbo] 0: num_leaves=89; min_data_in_leaf=3885 : y = 0.929 : 245.7 secs : initdesign

[mbo] 0: num_leaves=163; min_data_in_leaf=5122 : y = 0.931 : 254.5 secs : initdesign

[mbo] 0: num_leaves=51; min_data_in_leaf=2479 : y = 0.933 : 515.0 secs : initdesign

[mbo] 0: num_leaves=104; min_data_in_leaf=187 : y = 0.933 : 243.7 secs : initdesign

[mbo] 0: num_leaves=250; min_data_in_leaf=2015 : y = 0.935 : 692.5 secs : initdesign

[mbo] 0: num_leaves=8; min_data

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [38]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]

print(PARAM$out$lgbm$mejores_hiperparametros)

   num_leaves min_data_in_leaf num_iterations
        <int>            <int>          <int>
1:        250             2015           1112


## 1.5 Produccion

Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

##### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [39]:
PARAM$trainingstrategy$final_train <- c( 202107,
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  free_raw_data= TRUE
)

nrow( dfinal_train) # verifico el tamaño

[1] 240562

##### Final Training Hyperparameters

In [40]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

##### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [41]:
final_model <- lgb.train(
  data= dfinal_train,
  param= param_final,
  verbose= -100
)

In [42]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(final_model, "modelo.txt")

In [43]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(final_model))
archivo_importancia <- "impo.txt"

fwrite( tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

In [45]:
head(tb_importancia)

Feature,Gain,Cover,Frequency
<chr>,<dbl>,<dbl>,<dbl>
ctrx_quarter,0.07614613,0.0075082871,0.0128982639
rf_005_001,0.05046535,0.0020249489,0.0017865297
mcuentas_saldo,0.01583213,0.0082342240,0.0102304864
rf_010_005,0.01517279,0.0006315357,0.0007290323
cpayroll_trx,0.01387905,0.0027311636,0.0015141440
rf_010_003,0.01326359,0.0003275357,0.0002082949


#### Scoring

Aplico el modelo final a los datos del futuro

In [46]:
PARAM$trainingstrategy$future <- c(202109)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [47]:
# aplico final_model   a dfuture

prediccion <- predict(
  final_model,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

##### Tabla Prediccion

In [48]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion]

# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

## 1.6 Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle
<br>El notebook esta preparado para la Modalidad Gerencial

In [49]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "labo-i-2025-virtual-gerencial"
PARAM$kaggle$cortes <- seq(800, 1300, by = 50)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marco los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 


In [50]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

write_yaml( PARAM, file="PARAM.yml")

Loading required package: yaml



In [51]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Thu Nov 27 11:53:40 PM 2025"